In [1]:
import time
import threading
from datetime import datetime
from typing import Dict, Optional
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import BarData

In [3]:
class TradingApp(EClient, EWrapper):

    def __init__(self) -> None:

        EClient.__init__(self, self)
        self.data: Dict[int, pd.DataFrame] = {}
        self.nextOrderId: Optional[int] = None

    def error(self, reqId: int, errorCode: int, errorString: str, advanced: any = None) -> None:

        print(f"Error: {reqId}, {errorCode}, {errorString}")

    def nextValidId(self, orderId: int) -> None:

        super().nextValidId(orderId)
        self.nextOrderId = orderId

    def get_historical_data(self, reqId: int, contract: Contract) -> pd.DataFrame:

        self.data[reqId] = pd.DataFrame(columns=["time", "high", "low", "close"])
        self.data[reqId].set_index("time", inplace=True)
        self.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime="",
            durationStr="1 D",
            barSizeSetting="1 min",
            whatToShow="MIDPOINT",
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[],
        )
        time.sleep(5)
        return self.data[reqId]

    def historicalData(self, reqId: int, bar: BarData) -> None:

        df = self.data[reqId]

        df.loc[
            pd.to_datetime(bar.date, unit="s"), 
            ["open", "high", "low", "close", "volume"]
        ] = [bar.open, bar.high, bar.low, bar.close, bar.volume]

        df = df.astype(float)

        self.data[reqId] = df

    @staticmethod
    def get_contract(symbol: str) -> Contract:

        contract = Contract()
        contract.symbol = symbol
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        return contract

    def place_order(self, contract: Contract, action: str, order_type: str, quantity: int) -> None:

        order = Order()
        order.action = action
        order.orderType = order_type
        order.totalQuantity = quantity

        self.placeOrder(self.nextOrderId, contract, order)
        self.nextOrderId += 1
        print("Order placed")

In [4]:
app = TradingApp()

app.connect("127.0.0.1", 7497, clientId=5)

threading.Thread(target=app.run, daemon=True).start()

while True:
    if isinstance(app.nextOrderId, int):
        print("connected")
        break
    else:
        print("waiting for connection")
        time.sleep(1)


nvda = TradingApp.get_contract("NVDA")

waiting for connection
Error: -1, 2104, Market data farm connection is OK:usfarm.nj
Error: -1, 2104, Market data farm connection is OK:eufarm
Error: -1, 2104, Market data farm connection is OK:usopt.nj
Error: -1, 2104, Market data farm connection is OK:cashfarm
Error: -1, 2104, Market data farm connection is OK:usfarm
Error: -1, 2106, HMDS data farm connection is OK:ushmds
Error: -1, 2158, Sec-def data farm connection is OK:secdefnj
connected


In [5]:
nvda

4603203264: 0,NVDA,STK,,0.0,,,SMART,,USD,,,False,,combo:

In [6]:
data = app.get_historical_data(99, nvda)
data

,high,low,close,open,volume
time,,,,,
2025-09-30 08:00:00,181.40,180.60,180.95,181.01,-1.0
2025-09-30 08:01:00,181.02,180.86,180.92,180.95,-1.0
2025-09-30 08:02:00,180.97,180.80,180.87,180.92,-1.0
2025-09-30 08:03:00,180.92,180.78,180.91,180.87,-1.0
2025-09-30 08:04:00,180.92,180.60,180.66,180.91,-1.0
...,...,...,...,...,...
2025-09-30 12:49:00,182.74,182.68,182.74,182.72,-1.0
2025-09-30 12:50:00,182.77,182.72,182.76,182.74,-1.0
2025-09-30 12:51:00,182.79,182.73,182.78,182.76,-1.0
